In [1]:
import sys
sys.path.append("../")
import numpy as np
from sklearn.metrics import mean_squared_error as rmse, classification_report
import h5py
from IPython.display import display
import pandas as pd
from statistics import stdev

## ICFormer

In [11]:
import ICFormer

model = ICFormer(
        look_back=5,
        n_features=128,
        num_transformer_blocks=4,
        num_heads=2,
        head_size=28,
        ff_dim=28,
        mlp_units=128,
        mlp_dropout=0.0,
        dropout=0.0
    )

model.build(input_shape=(None, 5, 128))
model.load_weights('../saved/ICFormer_bestModel.hdf5')

In [7]:
def get_pred(model, time_steps, x_tests, y_test):
    '''
    Prints predictions for test sets
    Parameters
    ----------
    model: h5py object, trained model
    x_test: list, test sets
    y_test: array, test set labels
    reshape: bool, if True, an extra dimension is added to the input
    DTW: bool, if True, the data is reshaped to the DTW model input shape
    '''
    average = []
    std = []

    for x, y in zip(x_tests, y_test):
        cycles = [200, 400, 600, 800, 1000]
        
        predictions_reg, predictions_clf, attention_outputs, attention_weights, attention_outputs_sum = model.predict(x)
        predictions_reg = predictions_reg.reshape(predictions_reg.shape[0], time_steps, predictions_reg.shape[1]//time_steps)

        predictions_LLI = np.zeros(len(cycles))
        predictions_LAMPE = np.zeros(len(cycles))
        predictions_LAMNE = np.zeros(len(cycles))

        for cycle in range(x.shape[1]):

            predictions_LLI[cycle] = rmse(y[:, cycle, 0], predictions_reg[:, cycle, 0], squared=False)
            predictions_LAMPE[cycle] = rmse(y[:, cycle, 1], predictions_reg[:, cycle, 1], squared=False)
            predictions_LAMNE[cycle] = rmse(y[:, cycle, 2], predictions_reg[:, cycle, 2], squared=False)

        df = pd.DataFrame(np.stack((predictions_LLI, predictions_LAMPE, predictions_LAMNE)), index=['LLI', 'LAMPE', 'LAMNE'],columns=[200, 400, 600, 800, 1000])
        average.append(np.mean(df.mean(axis=1)))
        std.append(stdev(df.mean(axis=1)))		
        display(df)
        print('Average RMSPE: ', np.mean(df.mean(axis=1)))
        print('Std RMSPE: ', stdev(df.mean(axis=1)),'\n')
	
    print('Average RMSPE: ', np.mean(average))
    print('Std RMSPE: ', np.mean(std))

def get_pred_clf(model, x_tests, y_test):

    for x, y in zip(x_tests, y_test):
        
        predictions_reg, predictions_clf, attention_outputs, attention_weights, attention_outputs_sum = model.predict(x)
        print(classification_report(y, np.round(predictions_clf)))

In [5]:
chemistry = 'LFP' # 'LFP', 'NCA' or 'NMC'
path = '../data/'+chemistry

# load data
with h5py.File(path+'/test/datasets.h5', 'r') as h5f:
    test_datasets_ICs = h5f['x_test'][:]
    test_datasets_degradation_modes = h5f['y_test'][:]
    test_datasets_QLs = h5f['y_test_QLs'][:]
    test_datasets_silent_modes = h5f['y_test_silent_modes'][:]

The first table correspond to the training cell, the rest to the test cells

In [9]:
time_steps = 5
get_pred(model, time_steps, test_datasets_ICs, test_datasets_degradation_modes)

32/32 [==============================] - 2s 15ms/step


,200,400,600,800,1000
LLI,0.957082,1.021810,1.205021,1.537135,2.197389
LAMPE,1.259951,1.613073,2.379413,3.262489,4.206336
LAMNE,1.684896,2.091862,2.643165,3.316923,4.191695


Average RMSPE:  2.2378826375946694
Std RMSPE:  0.7495416046350268 

32/32 [==============================] - 0s 12ms/step


,200,400,600,800,1000
LLI,1.050230,1.110513,1.295199,1.612567,2.271381
LAMPE,1.407387,1.705058,2.420092,3.258179,4.187780
LAMNE,1.746784,2.137288,2.652703,3.296386,4.148494


Average RMSPE:  2.2866692769900427
Std RMSPE:  0.7160689816659415 

32/32 [==============================] - 0s 14ms/step


,200,400,600,800,1000
LLI,1.000920,1.063329,1.238386,1.548885,2.192205
LAMPE,1.356027,1.728296,2.548576,3.487995,4.493353
LAMNE,1.838117,2.324545,2.945684,3.709088,4.703200


Average RMSPE:  2.4119070389262802
Std RMSPE:  0.8894344998293058 

32/32 [==============================] - 0s 13ms/step


,200,400,600,800,1000
LLI,1.033370,1.084326,1.252086,1.585240,2.276062
LAMPE,1.527178,1.771069,2.461853,3.297517,4.229786
LAMNE,1.872128,2.324628,2.915471,3.650323,4.612412


Average RMSPE:  2.392896704030162
Std RMSPE:  0.8460088636825267 

Average RMSPE:  2.332338914385289
Std RMSPE:  0.8002634874532002


In [10]:
get_pred_clf(model, test_datasets_ICs, test_datasets_silent_modes)

32/32 [==============================] - 1s 16ms/step
              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96       753
         1.0       0.96      0.78      0.86       247

    accuracy                           0.94      1000
   macro avg       0.94      0.88      0.91      1000
weighted avg       0.94      0.94      0.93      1000

32/32 [==============================] - 0s 12ms/step
              precision    recall  f1-score   support

         0.0       0.95      0.97      0.96       753
         1.0       0.90      0.85      0.87       247

    accuracy                           0.94      1000
   macro avg       0.93      0.91      0.92      1000
weighted avg       0.94      0.94      0.94      1000

32/32 [==============================] - 1s 21ms/step
              precision    recall  f1-score   support

         0.0       0.95      0.98      0.96       753
         1.0       0.93      0.85      0.89       247

    accuracy        